# Mike Babb
# babbm@uw.edu
# Find anagrams
## Part 2: Generate and store the anagrams v2.0

In [1]:
# standard libraries - installed by default
import collections
import datetime
from itertools import product
import pickle
import sqlite3
import string
import os
import timeit

In [2]:
# external libraries - not installed by default
import numpy as np
import pandas as pd

In [3]:
from part_00_file_db_utils import *
from part_00_process_functions import *

### set input and output paths

In [4]:
# base file path
base_file_path = '/project/finding_anagrams'

In [5]:
# input path
in_file_path = 'data'
in_file_path = os.path.join(base_file_path, in_file_path)

In [6]:
# output db path and name
db_path = 'db'
db_path = os.path.join(base_file_path, db_path)

In [7]:
if os.path.exists(db_path):
    pass
else:
    os.makedirs(db_path)

In [8]:
db_name = 'words.db'

### process control flags

In [9]:
# max number of letters to slice to use for the generation of sub-matrices for
# options 5 and 6. More letters means more sub-matrices
# 3 seems to be the sweet spot
n_subset_letters = 3

# set write_data to True to store the generated list of anagrams
write_data = False

In [10]:
# start a timer to record the entire operation
total_time_start = datetime.datetime.now()

### load input data

In [11]:
word_df, wg_df, letter_dict, char_matrix, word_group_id_list, word_id_list, wchar_matrix = load_input_data(db_path = db_path, db_name = db_name, in_file_path = in_file_path)

...loading words into a dataframe...
...query execution took: 1.685907 seconds...
...loading word groups into a dataframe...
...query execution took: 1.923813 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


# Test the different matrix splitting techniques

In [12]:
# There are 6 different matrix extraction techniques:
# Option 1: No sub-matrices
# Option 2: Word-length
# Option 3: First letter
# Option 4: Single-least common letter
# Option 5: n least common letters
# Option 6: word-length and n least common letters
# See split_matrix() for a more elaborate description. 
# test_matrix_extraction_option() invokes the split_matrix() function 7 times to demonstrate the
# differences in timing and split_matrix() function output. 
# The table below showcases which objects are populated by different matrix extraction options

| Is object populated by matrix extraction technique?|
|----------------------------------------------------|  


| Object                      | 0 | 1 | 2 | 3 | 4 | 5 | 6 |
|----------------------------:|---|---|---|---|---|---|---|
| n_char_matrix_dict          | Y | N | Y | N | N | N | N |
| single_letter_matrix_dict   | Y | N | N | Y | Y | N | N |
| letter_selector_matrix_dict | Y | N | N | N | N | Y | N |
| nc_ls_matrix_dict           | Y | N | N | N | N | N | Y |


In [13]:
test_matrix_extraction_option(wg_df = wg_df,
                                  letter_dict = letter_dict,
                                  word_group_id_list = word_group_id_list,
                                  wchar_matrix = wchar_matrix,
                                  n_subset_letters = n_subset_letters)

*** matrix extraction option: 0 ***
...enumerating 16,101 records...
...1,000 records enumerated...
...2,000 records enumerated...
...3,000 records enumerated...
...4,000 records enumerated...
...5,000 records enumerated...
...6,000 records enumerated...
...7,000 records enumerated...
...8,000 records enumerated...
...9,000 records enumerated...
...10,000 records enumerated...
...11,000 records enumerated...
...12,000 records enumerated...
...13,000 records enumerated...
...14,000 records enumerated...
...15,000 records enumerated...
...16,000 records enumerated...
...16,101 sub-matrices created...
Total extraction time: 44.4 seconds.
n_char_matrix_dict: <class 'dict'>
single_letter_matrix_dict: <class 'dict'>
letter_selector_matrix_dict: <class 'dict'>
nc_ls_matrix_dict: <class 'dict'>
    word  lcase  n_chars first_letter  word_id  word_group_id letter_group  \
0      A      a        1            a        0              0            a   
1     aa     aa        2            a        1

In [14]:
# scrolling through this output, it looks options 1-5 take about 10 seconds while
# options 0 and 6 take about 60 seconds. We'll address that later through some profiling and
# using snakeviz. 

In [15]:
# Leaving the matrix extracton blank defaults to option 0: prepare all outputs
wg_df, n_char_matrix_dict, single_letter_matrix_dict, letter_selector_matrix_dict, nc_ls_matrix_dict= split_matrix(
    letter_dict = letter_dict,
    word_group_id_list = word_group_id_list,
        wg_df = wg_df,
    wchar_matrix = wchar_matrix, 
    n_subset_letters = n_subset_letters    
)

...enumerating 16,101 records...
...1,000 records enumerated...
...2,000 records enumerated...
...3,000 records enumerated...
...4,000 records enumerated...
...5,000 records enumerated...
...6,000 records enumerated...
...7,000 records enumerated...
...8,000 records enumerated...
...9,000 records enumerated...
...10,000 records enumerated...
...11,000 records enumerated...
...12,000 records enumerated...
...13,000 records enumerated...
...14,000 records enumerated...
...15,000 records enumerated...
...16,000 records enumerated...
...16,101 sub-matrices created...
Total extraction time: 45.33 seconds.


In [16]:
# Compute the search space for each matrix extraction option.
# The search space is how many candidates have to be evaluated
# when finding parent/child word relationships. 
# The smaller the search space for a given word, the faster the
# actual relationship determination. 
# compute_lookups() calculates this using the number of rows in each of the different matrices and sub-matrices

In [17]:
wg_lu_df = compute_lookups(wg_df=wg_df,
                    n_char_matrix_dict = n_char_matrix_dict,
                    single_letter_matrix_dict = single_letter_matrix_dict,
                    letter_selector_matrix_dict = letter_selector_matrix_dict,
                    nc_ls_matrix_dict = nc_ls_matrix_dict,
                          db_path = db_path, db_name = db_name)

...now writing: word_group_lookup_counts


# demonstrate the different matrix extraction options using the word 'achiever'

In [18]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,letter_selector,first_letter_id,single_letter_id,letter_selector_id,nc_ls_id
0,A,a,1,a,0,0,a,a,1,a,0,0,0,0
1,aa,aa,2,a,1,1,a,a,1,a,0,0,0,1
2,aal,aal,3,a,2,2,al,la,2,la,0,11,1081,2
3,all,all,3,a,5394,5305,al,la,1,la,0,11,1081,2
4,aalii,aalii,5,a,3,3,ail,lai,1,lai,0,11,1083,3


In [19]:
demo_word = 'achiever'

wg_id = word_df.loc[word_df['lcase'] == demo_word, 'word_group_id'].iloc[0]

demo_wg_df = wg_df.loc[wg_df['word_group_id'] == wg_id, : ]

# option 1 - Full matrix
# No additional data needed

# option 2 -  Number of characters
n_char = demo_wg_df['n_chars'].iloc[0]

# option 3 - First letter
first_letter_id = demo_wg_df['first_letter_id'].iloc[0]

# option 4 - Least common letter
single_letter_id = demo_wg_df['single_letter_id'].iloc[0]

# option 5 - Multiple least common letters
letter_selector_id = demo_wg_df['letter_selector_id'].iloc[0]

# option 6 - Number of characters and multiple least common letters
nc_ls_id = demo_wg_df['nc_ls_id'].iloc[0]


## Demontrate that the different matrix extraction options return identical results

### Select on the full matrix: option 1

In [20]:
# demo the full matrix selection
output = get_values_full_matrix(wg_id = wg_id, 
                    wchar_matrix = wchar_matrix,
                   word_group_id_list = word_group_id_list)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


### Select on the matrices split by word-length: option 2

In [21]:
# demo the n char selection
output = get_values_n_char(wg_id = wg_id,
                      n_char = n_char,
                      n_char_matrix_dict = n_char_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


### Select on the matrices split by the first letter: option 3

In [22]:
# demo the first letter selection
output = get_values_single_letter(wg_id = wg_id, single_letter_id = first_letter_id,
                                                           single_letter_matrix_dict = single_letter_matrix_dict)         

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


### Select on the matrices split by the single least common letter: option 4

In [23]:
# demo the first letter selection
output = get_values_single_letter(wg_id = wg_id, single_letter_id = single_letter_id,
                                                           single_letter_matrix_dict = single_letter_matrix_dict)         

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


### Select on the matrices split by the letter selector: option 5

In [24]:
# demo with the letter selector
output = get_values_letter_selector(wg_id = wg_id,
                      letter_selector_id = letter_selector_id,
                      letter_selector_matrix_dict = letter_selector_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


### Select on the matrices split by word-length and the letter selector: option 6

In [25]:
# demo with the n_char letter selector
output = get_values_n_char_letter_selector(wg_id = wg_id,
                           nc_ls_id = nc_ls_id,                           
                           nc_ls_matrix_dict=nc_ls_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1348               achiever
12440          archdeceiver
12445         archdetective
12624          architective
12737    archrepresentative
Name: lcase, dtype: object
The last five words are:
224512       urethrovesical
225856        vaucheriaceae
225857       vaucheriaceous
227047    vibrotherapeutics
233655       zepharovichite
Name: lcase, dtype: object


In [26]:
# we've tested with one word, let's time many evaluations to get a sense of how quickly 
# the different matrix extraction options work
# use the timeit() function to evaluate how long, on average, a single matrix operation
# takes to complete
# we do this by writing python code encapsulated in quotes which is then sent to the function
# we can store the quoted code in a dictionary and then enumerate. 
# we'll run each code chunk 100 times and then compute the average

n_trials = 100

code_snippet_dict = {
    'Matrix Selection Option 1: Selecting by full matrix':
"""get_values_full_matrix(wg_id = wg_id, wchar_matrix = wchar_matrix, word_group_id_list = word_group_id_list)""",
    'Matrix Selection Option 2: Selecting by word length':
"""get_values_n_char(wg_id = wg_id, n_char = n_char, n_char_matrix_dict = n_char_matrix_dict)""",
    'Matrix Selection Option 3: Selecting by first letter':
"""get_values_single_letter(wg_id = wg_id, single_letter_id = first_letter_id, single_letter_matrix_dict = single_letter_matrix_dict)""",    
    'Matrix Selection Option 4: Selecting by single least common letter':
"""get_values_single_letter(wg_id = wg_id, single_letter_id = single_letter_id, single_letter_matrix_dict = single_letter_matrix_dict)""",
    'Matrix Selection Option 5: Selecting by letter selector':
"""get_values_letter_selector(wg_id = wg_id, letter_selector_id = letter_selector_id, letter_selector_matrix_dict = letter_selector_matrix_dict)""",
    'Matrix Selection Option 6: Selecting by word length and letter selector':
"""get_values_n_char_letter_selector(wg_id = wg_id, nc_ls_id = nc_ls_id, nc_ls_matrix_dict=nc_ls_matrix_dict)"""
}

timing_list = []
for csd, cs in code_snippet_dict.items():    

    # here we time the code execution
    total_time = timeit.timeit(cs, number=n_trials, globals=globals())
    timing_list.append([csd, total_time])

    # total time    
    total_time_formatted = '{:,}'.format(round(total_time, 2))    

    # average time
    avg_time = total_time / n_trials
    avg_time_formatted = '{:,}'.format(round(avg_time, 2)) 
        
    print(csd)
    # average number of seconds per trial
    print('Total time:', total_time_formatted, 'seconds. Average time:', avg_time_formatted, 'seconds.')

    # add a line break to make it easier to read
    print()
    
    

Matrix Selection Option 1: Selecting by full matrix
Total time: 2.4472 seconds. Average time: 0.024472 seconds.

Matrix Selection Option 2: Selecting by word length
Total time: 2.0481 seconds. Average time: 0.020481 seconds.

Matrix Selection Option 3: Selecting by first letter
Total time: 1.6545 seconds. Average time: 0.016545 seconds.

Matrix Selection Option 4: Selecting by single least common letter
Total time: 0.25 seconds. Average time: 0.0025 seconds.

Matrix Selection Option 5: Selecting by letter selector
Total time: 0.0158 seconds. Average time: 0.000158 seconds.

Matrix Selection Option 6: Selecting by word length and letter selector
Total time: 0.0147 seconds. Average time: 0.000147 seconds.



In [27]:
# the combination of the word length and letter selector is the fastest

In [28]:
# these numbers are orders of magnitude different and therefore hard to interpret.
# let's rescale.

In [29]:
# we'll use product() to compute the cartersian product of the list of timings
# from the timing list, we can compute the ratio of one timing to another
# we can then build a dataframe and cross-tab

expanded_timing_list = []
for me_source, me_target in product(timing_list, repeat=2):    
    # let's unpack this
    me_source_option, me_source_timing = me_source
    me_target_option, me_target_timing = me_target

    me_source_option = me_source_option[17:25]
    me_target_option = me_target_option[17:25]
    me_source_target_timing_ratio = me_source_timing / me_target_timing
    
    # add to the list
    expanded_timing_list.append([me_source_option, me_target_option, me_source_timing, me_target_timing, me_source_target_timing_ratio])    

In [30]:
col_names = ['meo_source', 'meo_target', 'meo_source_timing', 'meo_target_timing', 'meo_timing_ratio']

In [31]:
timing_df = pd.DataFrame(data = expanded_timing_list, columns = col_names)

In [32]:
# use the pd.pivot_table() function to display the ratios

In [33]:
# we computed the product so all cells will be filled
# but we only need to look at the top-diagonal
# from left to right: we can see how much faster option 5 is than option 1
# or how much faster option 4 is than option 3
timing_table = timing_df.pivot_table(index = ['meo_source'], columns = ['meo_target'],
                           values =['meo_timing_ratio']).reset_index()

In [34]:
timing_table

meo_source meo_timing_ratio                                \
meo_target                    Option 1  Option 2  Option 3  Option 4   
0            Option 1         1.000000  1.194855  1.479058  9.788773   
1            Option 2         0.836922  1.000000  1.237856  8.192437   
2            Option 3         0.676106  0.807848  1.000000  6.618247   
3            Option 4         0.102158  0.122064  0.151097  1.000000   
4            Option 5         0.006475  0.007737  0.009577  0.063385   
5            Option 6         0.006023  0.007197  0.008909  0.058959   

                                    
meo_target    Option 5    Option 6  
0           154.432836  166.027138  
1           129.248205  138.951729  
2           104.412944  112.251920  
3            15.776527   16.960976  
4             1.000000    1.075077  
5             0.930166    1.000000

In [35]:
# wow. So, based on this analysis of the different 
# matrix extraction options using the word 'achiever':
# option 5 and 6 are the fastest, each over 100 times faster than option 1
# option 4 - just using the single least common letter - is 10X faster than option 1
# and 6X faster than option 3 - using the starting the letter of the word. 
